In [ ]:
%run "./Setup.ipynb"

In [ ]:
inputPath = "E:\\PySpark\\data\\users.json"

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df1 = spark.read.json(inputPath)
df1.show()

In [ ]:
df1.createOrReplaceTempView("users")

In [ ]:
spark.catalog.listTables()

In [ ]:
qry = """select age, count(1) as count
        from users
        where age is not null
        group by age
        order by count
        limit 5"""

df2 = spark.sql(qry)

df2.show()

#### Local Temp Views

In [ ]:
spark2 = spark.newSession()

In [ ]:
df3 = spark2.read.json(inputPath)

In [ ]:
spark.catalog.listTables()

In [ ]:
df3.createOrReplaceTempView("users2")

In [ ]:
spark2.catalog.listTables()

In [ ]:
spark2.catalog.dropTempView("users")

In [ ]:
qry = """select age, count(1) as count
        from users2
        where age is not null
        group by age
        order by count
        limit 5"""

df2 = spark2.sql(qry)

df2.show()

In [ ]:
spark.sql(qry).show()

In [ ]:
spark.catalog.currentDatabase()

#### GlobalTempViews

In [ ]:
df1.show()

In [ ]:
df1.createOrReplaceGlobalTempView("gusers")

In [ ]:
spark.catalog.listTables("global_temp")

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
qry2 = """select age, count(1) as count
        from global_temp.gusers
        where age is not null
        group by age
        order by count
        limit 5"""

df3 = spark.sql(qry2)

df3.show()

In [ ]:
spark2.sql(qry2).show()

#### Create a new session

In [ ]:
spark2 = spark.newSession()

In [ ]:
spark2.catalog.listTables()

In [ ]:
spark.catalog.listTables()

In [ ]:
print(qry2)

In [ ]:
df3 = spark2.sql(qry2)

df3.show()

#### Saving to tables

In [ ]:
spark.conf.get("spark.sql.warehouse.dir")

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
spark.sql("create database demodb")

In [ ]:
spark.catalog.listDatabases()

In [ ]:
df2 = df1.where("age is not null and phone is not null")
df2.show()

In [ ]:
df2.write.format("csv").option("header", True).saveAsTable("users_csv")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("select * from users_csv limit 5").show()

In [ ]:
spark.sql("use demodb")

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
df2.write.format("json").saveAsTable("users_json")

In [ ]:
df2.rdd.getNumPartitions()

In [ ]:
spark.sql("select * from users_json limit 5").show()

In [ ]:
spark.sql("select * from default.users_csv limit 5").show()

In [ ]:
spark.catalog.listTables('default')

In [ ]:
external_path = "E:\\PySpark\\external"

In [ ]:
df2.write.format("parquet").option("path", external_path).saveAsTable("users_parq_ext")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("select * from users_parq_ext limit 5").show()

In [ ]:
spark.sql("drop table users_json")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("drop table default.users_csv")

In [ ]:
spark.catalog.listTables('default')

In [ ]:
spark.sql("drop table users_parq_ext")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("select * from users_parq_ext limit 5").show()

In [65]:
df2.show()

+---+------+-------+----------+------+
|age|gender|   name|     phone|userid|
+---+------+-------+----------+------+
| 25|  Male|  Satya|8501099876|     1|
| 28|Female| Sandra|8508899001|     4|
| 13|  Male| Sundar|8522233456|     6|
| 28|  Male|  Steve|8501085009|     7|
| 55|Female| Smriti|9246655498|     8|
| 36|Female|  Veena|8508888888|     9|
| 13|  Male| Samuel|8522235624|    11|
| 28|  Male|  Raghu|8501082222|    12|
| 55|Female|  Ramya|9246654658|    13|
| 31|Female| Hasina|8345213235|    14|
| 13|  Male|Murugan|8522209563|    16|
| 31|  Male| Mukesh|8597638421|    17|
+---+------+-------+----------+------+



In [66]:
df2.write.format("csv").option("header", True).partitionBy("gender").saveAsTable("users_part")

In [67]:
spark.catalog.listTables()

[Table(name='users_part', database='demodb', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='users', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [70]:
spark.sql("select * from users_part where gender='Male' and age > 25").show()

+---+------+----------+------+------+
|age|  name|     phone|userid|gender|
+---+------+----------+------+------+
| 28| Steve|8501085009|     7|  Male|
| 28| Raghu|8501082222|    12|  Male|
| 31|Mukesh|8597638421|    17|  Male|
+---+------+----------+------+------+

